$$\textrm{Joaquin Peñuela Parra}$$
$$\textrm{University of Los Andes}$$
$$\textrm{High Energy Physics Group: Phenomenology of Particles}$$

This code was written to be running in Docker. If you do not have a Docker inside hep-server2 please refer to: https://github.com/Phenomenology-group-uniandes/Tutoriales_Generales

In [1]:
import os, sys

def add_parent_lib_path(name="lq_zprime"):
    sys.path.append(sys.path[0].split(name)[0])
    
add_parent_lib_path()

In [2]:
Path_Leptoquarks_searches = os.path.dirname(os.path.dirname(os.path.realpath('XBG_Discriminator_Histograms.ipynb')))

In [3]:
from Uniandes_Framework.ml_tools import tools
from Uniandes_Framework.delphes_reader import root_analysis

import pandas as pd

from ROOT import TCanvas #Permite poner los histogramas
from ROOT import THStack #Permite graficar varios histogramas al mismo tiempo
from ROOT import TLegend #Permite poner legends cuando se sobrelapan histogramas
from ROOT import TLatex #Permite poner avisos en Latex en las graficas

Welcome to JupyROOT 6.22/06


In [4]:
channels = ["hadronic_dLQ", "hadronic_sLQ", "hadronic_non-resonant", "semileptonic_dLQ", "semileptonic_sLQ", "semileptonic_non-resonant"]
signals = ['LQ_LQ', 'Tau_LQ', 'Tau_Tau'] + ['LQ_LQ_wo_RHC', 'Tau_LQ_wo_RHC', 'Tau_Tau_wo_RHC']
Masses = [
    "1000" 
]

bkgs = ['ttbar', 'z_jets', 'w_jets', 'stop','ww', 'wz', 'zz']

In [5]:
zp_signals = ['zp_tau_tau_m1000_g1_5', 'ta_ta_m1000_g1_5']

In [6]:
try: os.mkdir('Histograms')
except: pass

def escape_braces(strings):
    escaped_strings = []
    for s in strings:
        #escaped_s = s.replace('#tau', 'tau_jet').replace('b', 'b_jet').replace('#Delta R', '#Delta{R}').replace('#Delta ', '#Delta').replace('#Delta#eta', '#Delta{#eta}').replace('#Delta#phi', '#Delta{#phi}')
        escaped_s = s.replace('#Delta R', '#Delta{R}').replace('#Delta ', '#Delta').replace('#DeltapT', '#Delta{pT}').replace('#Delta#eta', '#Delta{#eta}').replace('#Delta#phi', '#Delta{#phi}')
        escaped_strings.append("{}".format(escaped_s))
    return escaped_strings

# def escape_braces(strings):
#     escaped_strings = []
#     for s in strings:
#         escaped_s = s.replace('{', '{{').replace('}', '}}')
#         escaped_strings.append("{}".format(escaped_s))
#     return escaped_strings

for channel in channels:
    try: os.mkdir(f'Histograms/{channel}')
    except: pass    
    
    most_important_features = []
    with open(os.path.join(os.getcwd(), 'XGB_models', channel, f'Most_Important_Features.txt'), "r") as f:
        for line in f: most_important_features.append(line.strip())
    f.close()
        
    for Mass in Masses:
        try: os.mkdir(f'Histograms/{channel}/M{Mass}')
        except: pass
        
        bkg_dict = {}
        signal_dict = {}
        
        signal_dict['zp_tau_tau_m1000_g1_5'] = os.path.join(os.sep,"disco4","pheno_csv_files","Leptoquarks_Searches",'zp_tau_tau_m1000_g1_5',f"zp_tau_tau_m1000_g1_5_{channel}.csv")
        signal_dict['ta_ta_m1000_g1_5'] = os.path.join(os.sep,"disco4","pheno_csv_files","Leptoquarks_Searches",'ta_ta_m1000_g1_5',f"ta_ta_m1000_g1_5_{channel}.csv")
        
        path_model_mass =  os.path.join(os.getcwd(), 'XGB_models', channel, f'M{Mass}_XGB.joblib')
        bkg_and_signal_dict = bkg_dict | signal_dict
        path_folder_mass = os.path.join(os.getcwd(), 'Histograms', channel, f'M{Mass}')
        
        with root_analysis.Quiet(): tools.hist_discriminator(path_model= path_model_mass, csv_dict = bkg_and_signal_dict, path_to_save= path_folder_mass, best_features= escape_braces(most_important_features), old_features = most_important_features)

[21:07:45] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[21:07:46] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[21:07:46] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, pl

In [7]:
!touch termino XGB_Discriminator_Histograms.txt